In [1]:
from jetbot import Robot
import sys
import glob
import serial
import time
import datetime

robot = Robot()

In [2]:
def serial_ports():

    """ Lists serial port names
        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
    """

    ports = glob.glob('/dev/ttyUSB*') 
    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result

In [9]:
def read_sensor_finite(reads):
    logfile = 'DGS_Log'
    device_identfier = ','
    output_response_time = 1
    baudrate = 9600
    serialport = ['','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','']
    serialdata = ['','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','']
    serial_port_list = []
    device_list = []


##################Ordering the serial ports based on device name##################
    clock = str(datetime.datetime.now())
    #print("Gathering ports list at " + clock)
    port_list = serial_ports()
    #print(port_list)
    
    i = 0
    while(i < len(port_list)):	
        serialport[i] =  serial.Serial(port_list[i], baudrate, timeout=5)
        i = i + 1
        #time.sleep(.5)#1
        
        i = 0
        while(i < len(port_list)):
            #print("port_list length: {}".format(len(port_list)))
            serialport[i].write('\r'.encode())
            #time.sleep(.5)#5 increase this if we add another sensor i think
            serialport[i].write('\r'.encode()) 
            serialdata[i]  = serialport[i].readline()
            time.sleep(output_response_time)
            i = i + 1
    i = 0
    while(i < len(port_list)):
        try:
            if device_identfier.encode() in serialdata[i]:
                serial_port_list.append(port_list[i])
                device_list.append(serialdata[i].decode('utf-8')[:12])
        except:

            pass

        i = i + 1 
    i = 0

    while(i < len(port_list)):
        serialport[i].close()
        i = i + 1
    #print(device_list)
    serial_port_list.sort(key=dict(zip(serial_port_list, device_list)).get)
    #print(serial_port_list)

##########################Serial ports opened in the new order and output read###########################

    i = 0

    while(i < len(serial_port_list)):
        serialport[i] =  serial.Serial(serial_port_list[i], baudrate, timeout=0)
        i = i + 1

    #time.sleep(1) #2
    clock = str(datetime.datetime.now())
    #print("Opening file for writing at " + clock)
    O3_sum = 0
    index = 0
    file = open(logfile, 'a') # append mode
    j = 0
    print("Reading", end ="")
    while(j < reads):
        i = 0
        while(i < len(serial_port_list)):
            serialport[i].write('\r'.encode())
            time.sleep(0.01)
            i = i + 1
        time.sleep(1) #5 This is how often to read from sensor (min is 1 second)
        i = 0
        while(i < len(serial_port_list)):
            serialdata[i]  = serialport[i].readline()
            i = i + 1
        clock = str(datetime.datetime.now())
        if device_identfier.encode() in serialdata[0]:
            file.write(clock + ', ')
            i = 0
            while(i < len(serial_port_list)):
                file.write(serialdata[i].decode('utf-8')[:-3] + ', ')
                sensor_data = serialdata[i].decode('utf-8')[:-3]   #SAMPLE: 121216010434, 63, 21, 33, 32610, 25588, 20602, 00, 00, 12, 45
                data_array = [x.strip() for x in sensor_data.split(',')]
                #print(sensor_data)
                O3 = int(data_array[2])   #index 5 is Raw Temp , 2 is temp, 1 is O3
                O3_sum = O3_sum + O3
                #print("Heat level: {}".format(O3))
                print("{}.".format(O3), end ="")
                i = i + 1

            file.write('\n')
            # time.sleep(55)
            j = j + 1
    #print("Logging data: " + logfile + ' ' + clock)

    file.close()

    i = 0 
    
    # close all open serial ports
    while(i < len(serial_port_list)):
        serialport[i].close()
        i = i + 1
#########################################################################################################
    """
    The format of the sensor_data output is:
    SN, PPB, T (°C), RH (%), ADC Raw, T Raw, RH Raw, Day, Hour, Minute, Second
    """
    avg_level = O3_sum / reads
    print("\nHeat level: {}".format(avg_level))
    return avg_level

In [4]:
start = time.time()
print("test")
end = time.time()
print(end - start)

test
0.0015408992767333984


In [11]:
def move_forward(distance):
    forward_x = .1
    forward_y = .1
    robot.set_motors(forward_x, forward_y)
    time.sleep(distance)
    robot.stop()
    
def move_back(distance):
    forward_x = -0.336
    forward_y = -0.3
    robot.set_motors(forward_x, forward_y)
    time.sleep(distance)
    robot.stop()   

In [12]:
move_forward(6)
move_back(6)

KeyboardInterrupt: 

In [10]:
level = []   # list of the previous levels
position = []# list of the previous positions the robot has traveled to

# Total reads to average at each point, 1 read per second so this is will be the time it spends at each point
reads = 60

# Distances to travel, anytime the robot changes the direction of its travel it 
#     will then lessen the distance it travel in future iterations of the loops
distance_1 = 6   # will move for 4 seconds
distance_2 = 3 # will move for 2 seconds
distance_3 = 2    # will move for 1 seconds

distance = distance_1    #initializing distance to the first distance to travel

last_move = 0    # 0 for forward, 1 for backward
move = 0         # 0 for forward, 1 for backward

old_level = 0
current_position = 0

i = 0 # index for the entire while loop

buffer = 15

start = time.time()

while 1:
    time.sleep(buffer)
    read = read_sensor_finite(reads)
    level.append(read)
    
    
    if level[i] >= old_level:   #new level is more or equal than last
        #keep going same direction
        if last_move == 0:
            move = 0
        else:
            move = 1     
    else:
        #change direction
        if last_move == 0:
            move = 1
        else:
            move = 0
        # and change the moving distance
        if distance == distance_1:
            distance = distance_2
        elif distance == distance_2:
            distance = distance_3
    
    last_move = move
    old_level = level[i]
    
    
    # moving the robot
    if move == 0: #move forward
        print("Moving forward " + str(distance) + " feet.")
        current_position = current_position + distance
        position.append(current_position)
        move_forward(distance)
        
    else:    #move backward
        print("Moving back " + str(distance) + " feet.")
        current_position = current_position - distance
        position.append(current_position)
        move_back(distance)


    # check how many times the robot has visited a spot and stop the test if it has been there 3 times
    count = 1
    if i > 0:    #starts during second position tests
        print("Positions: ", end = "")
        for x in range(len(position) - 1):   # test all postions but last because last is the current
            if position[x] == current_position:
                count = count + 1
            print(str(position[x]), end =" ")
        print("\nThis position is " + str(current_position) + " and has been visited " + str(count) + " times.")

        if count >= 3:
            break
    i = i + 1
    print("\n")

end = time.time()
trial_time = end - start

print("Source found at distance of " + str(current_position))
print("Trial took " + str(trial_time))

Reading21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.21.
Heat level: 21.0
Moving forward 6 feet.


Reading22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.22.
Heat level: 22.0
Moving forward 6 feet.
Positions: 6 
This position is 12 and has been visited 1 times.




KeyboardInterrupt: 

robot.stop()